In [1]:
#import modules
import pandas as pd
import pyarrow.parquet as pq # Used to read the data
import os 
import numpy as np
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import *

C:\Users\290002494\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#use this for my attention
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [3]:
#this is what the competition is judged on
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [4]:
# just load train data
df_train = pd.read_csv('metadata_train.csv')
# set index, it makes the data access much faster
df_train = df_train.set_index(['id_measurement', 'phase'])
df_train.head(6)

#signal_id 0,1,2 are the 3 that all relate to id_measurement 0

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1
               2              5       1

In [5]:
#read in the parquet files
train = pq.read_pandas('train.parquet').to_pandas()
train.shape

(800000, 8712)

In [6]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711
0,18,1,-19,-16,-5,19,-15,15,-1,-16,...,18,-22,12,8,13,6,-21,-15,-9,20
1,18,0,-19,-17,-6,19,-17,16,0,-15,...,17,-21,12,8,14,7,-19,-15,-8,21
2,17,-1,-20,-17,-6,19,-17,15,-3,-15,...,16,-21,13,8,15,8,-18,-14,-8,22
3,18,1,-19,-16,-5,20,-16,16,0,-15,...,16,-21,12,8,15,8,-19,-14,-7,23
4,18,0,-19,-16,-5,20,-17,16,-2,-14,...,17,-22,12,8,15,8,-18,-14,-8,23


In [7]:
#neural networks train faster on standardized data
max_num = 127
min_num = -128
def min_max_transf(ts, min_data, max_data, range_needed=(-1,1)):
    if min_data < 0:
        ts_std = (ts + abs(min_data)) / (max_data + abs(min_data))
    else:
        ts_std = (ts - min_data) / (max_data - min_data)
    if range_needed[0] < 0:    
        return ts_std * (range_needed[1] + abs(range_needed[0])) + range_needed[0]
    else:
        return ts_std * (range_needed[1] - range_needed[0]) + range_needed[0]

In [8]:
# This is one of the most important peace of code of this Kernel
# Any power line contain 3 phases of 800000 measurements, or 2.4 millions data 
# It would be praticaly impossible to build a NN with an input of that size
# The ideia here is to reduce it each phase to a matrix of <n_dim> bins by n features
# Each bean is a set of 5000 measurements (800000 / 160), so the features are extracted from this 5000 chunk data.
sample_size = 800000
def transform_ts(ts, n_dim=160, min_max=(-1,1)):
    # convert data into -1 to 1
    ts_std = min_max_transf(ts, min_data=min_num, max_data=max_num)
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        ts_range = ts_std[i:i + bucket_size]
        # calculate each feature
        mean = ts_range.mean()
        std = ts_range.std() # standard deviation
        std_top = mean + std # I have to test it more, but is is like a band
        std_bot = mean - std
        # I think that the percentiles are very important, it is like a distribuiton analysis from eath chunk
        percentil_calc = np.percentile(ts_range, [0, 1, 25, 50, 75, 99, 100]) 
        max_range = percentil_calc[-1] - percentil_calc[0] # this is the amplitude of the chunk
        relative_percentile = percentil_calc - mean # maybe it could heap to understand the asymmetry
        # now, we just add all the features to new_ts and convert it to np.array
        new_ts.append(np.concatenate([np.asarray([mean, std, std_top, std_bot, max_range]),percentil_calc, relative_percentile]))
    return np.asarray(new_ts)

In [9]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data(start, end):
    # load a piece of data from file
    X = []
    y = []
    # using tdqm to evaluate processing time
    # takes each index from df_train and iteract it from start to end
    # it is divided by 3 because for each id_measurement there are 3 id_signal, and the start/end parameters are id_signal
    for id_measurement in tqdm(df_train.index.levels[0].unique()[0:df_train.index.max()[0]]): #0 to 2903
        X_signal = []
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df_train.loc[id_measurement].loc[phase]
            # but just append the target one time, to not triplicate it
            if phase == 0:
                y.append(target)
            #for each signal id (not measurement id) do feature engineering
            X_signal.append(transform_ts(train[str(signal_id)]))
        # concatenate all the 3 phases so they are all in one row
        X_signal = np.concatenate(X_signal, axis=1)
        # add the data to X
        X.append(X_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [10]:
X = []
y = []
def load_all():
    total_size = len(df_train)
    for ini, end in [(0, total_size)]:
        X_temp, y_temp = prep_data(ini, end)
        X.append(X_temp)
        y.append(y_temp)
load_all()
X = np.concatenate(X)
y = np.concatenate(y)

100%|██████████████████████████████████████| 2903/2903 [12:40<00:00,  3.93it/s]


In [11]:
#x[0] is the number of id_measurements
#x[1] is the number of chunks of data
#x[2] is the number of features for each
print(X.shape, y.shape)

(2903, 160, 57) (2903,)


In [12]:
X[0,0,:]

array([ 0.14322353,  0.00713529,  0.15035882,  0.13608824,  0.04705882,
        0.12156863,  0.12941176,  0.1372549 ,  0.14509804,  0.14509804,
        0.16078431,  0.16862745, -0.0216549 , -0.01381176, -0.00596863,
        0.00187451,  0.00187451,  0.01756078,  0.02540392,  0.0085051 ,
        0.00690299,  0.01540809,  0.00160211,  0.05490196, -0.01960784,
       -0.00392157,  0.00392157,  0.01176471,  0.01176471,  0.01960784,
        0.03529412, -0.02811294, -0.01242667, -0.00458353,  0.00325961,
        0.00325961,  0.01110275,  0.02678902, -0.15020235,  0.00815639,
       -0.14204596, -0.15835875,  0.07058824, -0.18431373, -0.16862745,
       -0.15294118, -0.15294118, -0.14509804, -0.12941176, -0.11372549,
       -0.03411137, -0.0184251 , -0.00273882, -0.00273882,  0.00510431,
        0.02079059,  0.03647686])

In [13]:
#LSTM
def model_lstm(input_shape):
    inp = Input(shape=(input_shape[1], input_shape[2],))
    
    #bidirectional works the best when we are dealing with Attention
    x = Bidirectional(LSTM(128, return_sequences=True))(inp) #CuDNNLSTM is faster but needs a gpu
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    
    #ATTENTION GOES HERE
    x = Attention(input_shape[1])(x)
    
    #fully connect on the way out to help deal with nonlinear outputs
    x = Dense(64, activation = "relu")(x)
    
    #binary classification
    x = Dense(1, activation = "sigmoid")(x)
    
    model = Model(inputs = inp, outputs = x)
    
    #compile our model
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=[matthews_correlation])
    
    return model

In [14]:
model = model_lstm(X.shape)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 57)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 160, 256)          190464    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 160, 128)          164352    
_________________________________________________________________
attention_1 (Attention)      (None, 128)               288       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 363,425
Trainable params: 363,425
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#use this if you want to 5 fold cross validate
# First, create a set of indexes of the 5 folds. Shuffle is good in DL so you don't do correlated learning
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=2019).split(X, y))
preds_val = []
y_val = []
# Then, iteract with each fold
# If you dont know, enumerate(['a', 'b', 'c']) returns [(0, 'a'), (1, 'b'), (2, 'c')]
for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)
    # This checkpoint helps to avoid overfitting. It just save the weights of the model if it delivered an
    # validation matthews_correlation greater than the last one.
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit(train_X, train_y, batch_size=128, epochs=5, validation_data=[val_X, val_y], callbacks=[ckpt])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict(val_X, batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)[...,0]
y_val = np.concatenate(y_val)
print(preds_val.shape)
print(y_val.shape)

Beginning fold 1
Train on 2322 samples, validate on 581 samples
Epoch 1/5
2322/2322 [==============================] - ETA: 1:15 - loss: 0.6924 - matthews_correlation: -0.080 - ETA: 55s - loss: 0.6794 - matthews_correlation: -0.040 - ETA: 47s - loss: 0.6634 - matthews_correlation: -0.02 - ETA: 42s - loss: 0.6540 - matthews_correlation: -0.02 - ETA: 38s - loss: 0.6304 - matthews_correlation: -0.01 - ETA: 34s - loss: 0.6101 - matthews_correlation: -0.01 - ETA: 31s - loss: 0.5794 - matthews_correlation: -0.01 - ETA: 28s - loss: 0.5402 - matthews_correlation: -0.01 - ETA: 26s - loss: 0.5084 - matthews_correlation: -0.00 - ETA: 23s - loss: 0.4783 - matthews_correlation: -0.00 - ETA: 20s - loss: 0.4556 - matthews_correlation: -0.00 - ETA: 17s - loss: 0.4390 - matthews_correlation: -0.00 - ETA: 14s - loss: 0.4248 - matthews_correlation: -0.00 - ETA: 11s - loss: 0.4190 - matthews_correlation: -0.00 - ETA: 8s - loss: 0.4004 - matthews_correlation: -0.0053 - ETA: 6s - loss: 0.3985 - matthews_cor

In [46]:
#use this if you want to avoid cross validation
model.fit(X,y, epochs=10, batch_size = 128)

Epoch 1/10
2903/2903 [==============================] - ETA: 55s - loss: 0.3100 - matthews_correlation: 0.0000e+0 - ETA: 53s - loss: 0.2404 - matthews_correlation: 0.0000e+0 - ETA: 51s - loss: 0.2307 - matthews_correlation: 0.0000e+0 - ETA: 48s - loss: 0.2159 - matthews_correlation: 0.0000e+0 - ETA: 46s - loss: 0.2195 - matthews_correlation: 0.0000e+0 - ETA: 43s - loss: 0.2061 - matthews_correlation: 0.0000e+0 - ETA: 40s - loss: 0.2000 - matthews_correlation: 0.0000e+0 - ETA: 38s - loss: 0.2151 - matthews_correlation: 0.0000e+0 - ETA: 35s - loss: 0.2152 - matthews_correlation: 0.0000e+0 - ETA: 32s - loss: 0.2023 - matthews_correlation: 0.0000e+0 - ETA: 30s - loss: 0.2034 - matthews_correlation: 0.0000e+0 - ETA: 27s - loss: 0.2132 - matthews_correlation: 0.0000e+0 - ETA: 25s - loss: 0.2114 - matthews_correlation: 0.0000e+0 - ETA: 22s - loss: 0.2148 - matthews_correlation: 0.0000e+0 - ETA: 20s - loss: 0.2127 - matthews_correlation: 0.0000e+0 - ETA: 17s - loss: 0.2167 - matthews_correlati

In [ ]:
# The output of this kernel must be binary (0 or 1), but the output of the NN Model is float (0 to 1).
# So, find the best threshold to convert float to binary is crucial to the result
# this piece of code is a function that evaluates all the possible thresholds from 0 to 1 by 0.01
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.05 for i in range(100)]):
        score = K.eval(matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64)))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    return search_result

In [ ]:
best_threshold = threshold_search(y_val, preds_val)['threshold']

In [47]:
pred = model.predict(X, batch_size = 128)

In [67]:
predictions = []
for preds in pred:
    for i in range(3):
        predictions.append(preds)
np.array(predictions).shape

(8709, 1)

In [73]:
predictions = np.round(predictions,0)

(2903,)